## 実験3


In [ ]:
#(5)
#| echo: false
def counter2hms(c):
    H = 60*60
    M = 60
    h, c = c//H, c%H
    m, s = c//M, c%M
    return int(h),int(m),int(s)

In [ ]:
#(8)
#| label: fig_test_with_various_labeled_OvO
#| fig-cap: "Testing with Various Labeled Data Proportion (OvO)"

from qns3vm import QN_S3VM_OVO
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import random
from matplotlib import style
from matplotlib import pyplot as plt
style.use(['science','ieee'])
from time import sleep, perf_counter
from s3vm_pines.module import train_test_split3, labeled_unlabeled_sample, colored_map
from IndianPines import load, recategorize17to10_csv

pines = load(pca=20,recategorize_rule=recategorize17to10_csv)

train_test_status, train_test_status_name = train_test_split3()

rg = random.Random()

seed_l = None
seed_u = None

lam = 0.008765625
lamU = 1e-7 # 0に近いと完全教師

prop_train_l_list = [0.1, 0.2, 0.3, 0.4, 0.5]

acc = list()
err = list()
f1 = list()
fig6 = [0 for _ in prop_train_l_list]
ax6 = [0 for _ in prop_train_l_list] * 3

t_start = perf_counter()
t_old = t_start
for i, prop_train_l in enumerate(prop_train_l_list):
    print('='*3 + f'{prop_train_l = }' + '='*3)
    p_labeled = p_unlabeled = prop_train_l
    l_u_t_conf = dict(
        p_labeled = p_labeled,
        p_unlabeled = p_unlabeled,
        train_test_status = train_test_status,
        seed_labeled = seed_l,
        seed_unlabeled = seed_u
    )
    l_u_t_status, l_u_t_status_name = labeled_unlabeled_sample(**l_u_t_conf)

    for j, s in enumerate(l_u_t_status_name):
        n = l_u_t_status[l_u_t_status==j].shape[0]
        print(f'{j}:{s}:{n}')

    X_train_l = pines.features[l_u_t_status==3].tolist()
    L_train_l = pines.target[l_u_t_status==3].tolist()
    X_train_u = pines.features[l_u_t_status==4].tolist()
    L_train_u = pines.target[l_u_t_status==4].tolist()
    X_test = pines.features[l_u_t_status==1].tolist()
    L_test = pines.target[l_u_t_status==1].tolist()

    fig6[i] = plt.figure(figsize=(6.4*2,4.8*4), constrained_layout=True)
    # labeled
    ax6[3*i+0] = fig6[i].add_subplot(4,2,1)
    colored_map(ax6[3*i+0], L_train_l, pines.coordinates[l_u_t_status==3], recategorize_rule=recategorize17to10_csv, gt_gic=False)
    ax6[3*i+0].set_title(f'labeled area {prop_train_l =}')
    # unlabeled
    ax6[3*i+1] = fig6[i].add_subplot(4,2,3)
    colored_map(ax6[3*i+1], L_train_u, pines.coordinates[l_u_t_status==4], recategorize_rule=recategorize17to10_csv, gt_gic=False)
    ax6[3*i+1].set_title(f'unlabeled area {prop_train_l =}')


    clf = QN_S3VM_OVO(X_train_l, L_train_l, X_train_u, random_generator=rg, lam=lam, lamU=lauU)
    clf.train()

    preds = clf.predict(X_test)
    err_ = classification_error(preds,L_test)
    acc_ = accuracy_score(L_test, preds)
    f1_ = f1_score(L_test, preds, average="micro")

    ax6[3*i+2] = fig6[i].add_subplot(2,1,2)
    colored_map(ax6[3*i+2], preds, pines.coordinates[l_u_t_status==1], recategorize_rule=recategorize17to10_csv, gt_gic=False)
    ax6[3*i+2].set_title(f'predicted label for tested area {prop_train_l =}, {acc_ = }, {f1_ = }')

    fig6[i].tight_layout(rect=[0,0,1,0.96])
    fig6name = f'kotaro_exp03_fig06_{i:0d}.png'
    fig6[i].savefig(fig6name)

    err.append(err_)
    acc.append(acc_)
    f1.append(f1_)

    # counter
    t_now = perf_counter()
    t_lap = t_now-t_old
    t_ave = (t_now-t_start)/(i+1)
    t_rest = t_ave * (len(prop_train_l_list)-(i+1))
    ho,mo,so = counter2hms(t_old)
    hl,ml,sl = counter2hms(t_lap)
    hr,mr,sr = counter2hms(t_rest)
    print(f'lap {i+1} | {hl:2d}:{ml:2d}:{sl:2d} + {ho:2d}:{mo:2d}:{so:2d}, est {hr:2d}:{mr:2d}:{sr:2d}')
    t_old = t_now

fig7 = plt.figure()
ax7 = fig7.add_subplot(1,1,1)
ax7.plot(prop_train_l_list, acc, 'o', label='accuracy score')
ax7.plot(prop_train_l_list, f1, 's--', label='f1 score')
ax7.legend()
ax7.set_xlabel(r'Amount of Labeled Data [$\%$]')
ax7.set_ylabel(r'Score[$\%$]')
ax7.set_title('Testing with Various Labeled Data Proportion')
fig7.tight_layout(rect=[0,0,1,0.96])
fig7.savefig("kotaro_exp03_fig07.png")